In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from umap import UMAP
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()  # noqa
import dash_bio as dashbio
from python.cogtext.datasets.pubmed import PubMedDataLoader
from python.cogtext.similarity_matrix import get_similarity_matrix
from sklearn.preprocessing import normalize
from tqdm import tqdm

In [2]:
# load weights, clusters and metadata (takes ~ 20sec)

clusters = pd.read_csv('models/gpt3/abstracts_gpt3ada_clusters.csv.gz', index_col=0)
weights = np.load('models/gpt3/abstracts_gpt3ada_weights.npz')['arr_0']
clusters['weights'] = list(weights)

# load abstracts
pubmed = PubMedDataLoader(preprocessed=False, drop_low_occurred_labels=False).load()
pubmed = pubmed.merge(clusters, on='pmid', how='left')

pubmed.dropna(subset=['cluster'], inplace=True)

tasks = pubmed.query('category.str.contains("Task")')['label'].unique()
constructs = pubmed.query('category.str.contains("Task")')['label'].unique()

print(f'Successfully loaded a list of {len(pubmed)} topic-embeddings.')

Successfully loaded a list of 293014 topic-embeddings.


In [3]:
from sklearn.metrics.pairwise import pairwise_kernels

node_features = pubmed.groupby(['category','label'])['weights'].apply(np.mean)


def calc_similarity(a,b):
  return pairwise_kernels(a.reshape(1,-1), b.reshape(1,-1), metric='cosine')[0][0]

sim = node_features.apply(pd.Series).T.corr(method=calc_similarity)
sim

category                                           CognitiveConstruct  \
label                                                       Attention   
category           label                                                
CognitiveConstruct Attention                                 1.000000   
                   AttentionalControl                        0.752932   
                   AttentionalInertia                        0.295641   
                   BehavioralControl                         0.483556   
                   BehavioralRegulation                      0.430609   
...                                                               ...   
CognitiveTask      TowerOfHanoi                              0.613160   
                   TowerOfLondon                             0.558000   
                   TwoStep                                   0.085150   
                   Verbal_fluency_task                       0.610400   
                   WCST_-_Wisconsin_Card_Sort_Test           0.459221   

category                                                               \
label                                              AttentionalControl   
category           label                                                
CognitiveConstruct Attention                                 0.752932   
                   AttentionalControl                        1.000000   
                   AttentionalInertia                        0.245826   
                   BehavioralControl                         0.407276   
                   BehavioralRegulation                      0.325410   
...                                                               ...   
CognitiveTask      TowerOfHanoi                              0.511157   
                   TowerOfLondon                             0.445758   
                   TwoStep                                   0.056423   
                   Verbal_fluency_task                       0.467205   
                   WCST_-_Wisconsin_Card_Sort_Test           0.361333   

category                                                               \
label                                              AttentionalInertia   
category           label                                                
CognitiveConstruct Attention                                 0.295641   
                   AttentionalControl                        0.245826   
                   AttentionalInertia                        1.000000   
                   BehavioralControl                         0.129154   
                   BehavioralRegulation                      0.114072   
...                                                               ...   
CognitiveTask      TowerOfHanoi                              0.154857   
                   TowerOfLondon                             0.106378   
                   TwoStep                                   0.049944   
                   Verbal_fluency_task                       0.117599   
                   WCST_-_Wisconsin_Card_Sort_Test           0.068427   

category                                                              \
label                                              BehavioralControl   
category           label                                               
CognitiveConstruct Attention                                0.483556   
                   AttentionalControl                       0.407276   
                   AttentionalInertia                       0.129154   
                   BehavioralControl                        1.000000   
                   BehavioralRegulation                     0.530490   
...                                                              ...   
CognitiveTask      TowerOfHanoi                             0.264296   
                   TowerOfLondon                            0.215750   
                   TwoStep                                  0.032941   
                   Verbal_fluency_task                      0.23745

Cosine similarity of the construct vectors regardless of the tasks they rely on is as follows.

In [4]:
plot_data = sim.droplevel(0).droplevel(0,1).drop(index=tasks, columns=tasks, errors='ignore')

dashbio.Clustergram(
  data=plot_data,
  column_labels=plot_data.columns.to_list(),
  row_labels=plot_data.index.to_list(),
  cluster='all',
  center_values=False,
  height=800,
  width=900,
  display_ratio=[0.001, 0.001],
  color_map='turbo',
  # hidden_labels=['col'],
  row_dist='cosine',
  col_dist='cosine',
  # standardize='col'
)


In [96]:
# create a graph

import stellargraph as sg
from stellargraph.data import UniformRandomMetaPathWalk

sim.columns.name = 'source'
sim.index.name = 'target'

# keep = np.triu(np.ones(sim.shape)).astype('bool').reshape(sim.size)
adj = sim.droplevel(0).droplevel(0,1).stack()
adj.index.names = ['source', 'target']
adj = adj.to_frame().rename(columns={0:'weight'}).reset_index()

adj = adj.query('target != source')

# remove low-weighted edges
edges = adj.query('weight >= weight.median() + weight.std()')


tasks = node_features.apply(pd.Series).query('category == "CognitiveTask"').reset_index(0,drop=True)
constructs = node_features.apply(pd.Series).query('category == "CognitiveConstruct"').reset_index(0,drop=True)


# We are interested indirect task-construct links, so we need to
# filter out edges within the nodes of the same type.
edges['source_type'] = edges['source'].apply(lambda x: 'task' if x in tasks.index else 'construct')
edges['target_type'] = edges['target'].apply(lambda x: 'task' if x in tasks.index else 'construct')
edges = edges.query('source_type != target_type')

G = sg.StellarGraph(
  nodes = {'task': tasks,
           'construct': constructs},
  edges=edges[['source', 'target', 'weight']])

# recreate the graph with connected nodes only
connected_nodes = list(G.connected_components())[0]

G = sg.StellarGraph(
  nodes = {'task': tasks.query('index in @connected_nodes'),
           'construct': constructs.query('index in @connected_nodes')},
  edges=edges.query('source in @connected_nodes or target in @connected_nodes')[['source', 'target', 'weight']])

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 140, Edges: 2788

 Node types:
  task: [76]
    Features: float32 vector, length 509
    Edge types: task-default->construct
  construct: [64]
    Features: float32 vector, length 509
    Edge types: construct-default->task

 Edge types:
    construct-default->task: [2788]
        Weights: range=[0.297252, 0.960602], mean=0.443031, std=0.117672
        Features: none


Now we use Metapath2Vec for the graph embedding.

In [97]:
# %pip install csrgraph


import networkx as nx
import csrgraph as cg

def weighted_metapath_random_walk(graph,
                                  walk_length:int,
                                  n_walks:int,
                                  metapaths:list[str]):

  _g = cg.csrgraph(graph)
  
  walk_patterns = [mp[:-1] * (walk_length // (len(mp) - 1)) + mp[-1:0]
                   for mp in metapaths]
  
  valid_walks = []
  
  node_labels = np.array(graph.nodes())
  
  max_iter = 10000
  iter = 0

  while iter < max_iter:
    iter += 1
    walks = _g.random_walks(walk_length, n_walks, start_nodes=None)
    types = [[graph.nodes[node_labels[node]]['type'] for node in walk]
              for walk in walks]
    walks = [node_labels[walks[i]]
             for i,t in enumerate(types)
             if t in walk_patterns #and len(set(walks[i])) == len(walks[i])  # no loop
            ]
    valid_walks.extend(walks)
    valid_walks = pd.DataFrame(walks).drop_duplicates().values.tolist()

    if len(valid_walks) >= n_walks:
      return np.vstack(valid_walks[:n_walks]).tolist()

  return np.vstack(valid_walks).tolist()


metapaths = [
    ['task', 'construct', 'task'],
    ['construct', 'task', 'construct'],
]

walks = weighted_metapath_random_walk(G.to_networkx(node_type_attr='type'),
                                      walk_length=100,
                                      n_walks=1000,
                                      metapaths=metapaths)

print(f'Generated {len(walks)} random walks.')


Generated 1000 random walks.


In [140]:
from gensim.models import Word2Vec
model = Word2Vec(walks, vector_size=128, min_count=0, window=7, sg=1, workers=1, epochs=100)

model.save('models/gpt3/abstracts_metapath2vec.pkl')

# query the model
model.wv.most_similar('TMT_-_Trail_Making_Task')

[('LNS_-_Letter_Number_Sequencing', 0.5849911570549011),
 ('NBackTask', 0.5571386814117432),
 ('Verbal_fluency_task', 0.5498847961425781),
 ('Semantic_Fluency_test', 0.54936683177948),
 ('ContiniousPerformanceTask', 0.5465055108070374),
 ('Digit_Span', 0.5300901532173157),
 ('SimpleRT', 0.5285316109657288),
 ('RNG_-_Random_Number_Generation_task', 0.5265634059906006),
 ('Hayling_test', 0.47788915038108826),
 ('WCST_-_Wisconsin_Card_Sort_Test', 0.47529807686805725)]

In [128]:
from sklearn.metrics.pairwise import cosine_similarity

# model.wv.get_normed_vectors()
keys = model.wv.key_to_index.keys()
m2v_vectors = model.wv.vectors


m2v_plot_data = pd.DataFrame(cosine_similarity(m2v_vectors), index=keys, columns=keys)

# drop tasks
m2v_plot_data.drop(index=tasks.index, errors='ignore', inplace=True)
m2v_plot_data.drop(columns=tasks.index, errors='ignore', inplace=True)

dashbio.Clustergram(
  data=m2v_plot_data,
  column_labels=m2v_plot_data.columns.to_list(),
  row_labels=m2v_plot_data.index.to_list(),
  cluster='all',
  center_values=False,
  height=800,
  width=900,
  display_ratio=[0.001, 0.001],
  color_map='turbo',
  # hidden_labels=['col'],
  # row_dist='cosine',
  # col_dist='cosine',
  # standardize='col'
)